In [126]:
import os
import sys
import glob
import time
import datetime
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import random
import scipy as sp

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from functools import partial
from multiprocessing.pool import Pool
from time import time

# Read in CSVs | Add FID | Concatenate

In [231]:
file_zero = pd.read_csv("Songs/songs0.csv")
file_zero["fid"] = 0
file_one = pd.read_csv("Songs/songs1.csv")
file_one["fid"] = 1
complete_playlists = pd.DataFrame(np.concatenate([file_zero, file_one]), columns = file_zero.columns)

In [248]:
complete_playlists.head()

pos        artist_name                             track_uri  \
fid pid                                                                
0   0     0      Missy Elliott  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI   
    0     1     Britney Spears  spotify:track:6I9VzXrHxO9rA9A5euc8Ak   
    0     2            Beyoncé  spotify:track:0WqIKmW4BTrj3eJFmnCKMv   
    0     3  Justin Timberlake  spotify:track:1AWQoqb9bSvzTjaLralEkT   
    0     4             Shaggy  spotify:track:1lzr43nnXAijIGYnCT8M8H   

                                    artist_uri  \
fid pid                                          
0   0    spotify:artist:2wIVse2owClT7go1WT98tk   
    0    spotify:artist:26dSoYclwsYLMAKD3tpOr4   
    0    spotify:artist:6vWDO969PvNqNYHIOW5v0m   
    0    spotify:artist:31TPClRtHm23RisEBtV3X7   
    0    spotify:artist:5EvFsr3kj42KNv97ZEnqij   

                                         track_name  \
fid pid                                               
0   0    Lose Control (feat. Ciara & Fat Man Scoop)   
    0                                         Toxic   
    0                                 Crazy In Love   
    0                                Rock Your Body   
    0                                  It Wasn't Me   

                                    album_uri duration_ms  \
fid pid                                                     
0   0    spotify:album:6vV5UrXcfyQD1wu4Qo2I9K      226863   
    0    spotify:album:0z7pVBGOD7HCIB7S8eLkLI      198800   
    0    spotify:album:25hVFAxTlDvXbx2X2QkUkE      235933   
    0    spotify:album:6QPkyl04rXwTGlGlcYaRoW      267266   
    0    spotify:album:6NmFmPX56pcLBOFMhIiKvF      227600   

                                           album_name  
fid pid                                                
0   0                                    The Cookbook  
    0                                     In The Zone  
    0    Dangerously In Love (Alben für die Ewigkeit)  
    0                                       Justified  
    0                                        Hot Shot

# Set index to ["fid", "pid"]

In [ ]:
complete_playlists.set_index(["fid", "pid"], inplace=True)

# Get all unique songs, artists, and albums for Collaborative Filtering

In [251]:
complete_playlists['track_uri'].unique().shape

(57884,)

In [252]:
all_unique_songs.shape

(48774,)

In [255]:
all_unique_songs = complete_playlists['track_name'].unique()
all_unique_albums = complete_playlists['album_name'].unique()
all_unique_artists = complete_playlists['artist_name'].unique()

In [256]:
all_unique_albums.shape

(27393,)

In [257]:
all_unique_artists.shape

(14914,)

In [234]:
complete_playlists

pos            artist_name                             track_uri  \
fid pid                                                                    
0   0     0          Missy Elliott  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI   
    0     1         Britney Spears  spotify:track:6I9VzXrHxO9rA9A5euc8Ak   
    0     2                Beyoncé  spotify:track:0WqIKmW4BTrj3eJFmnCKMv   
    0     3      Justin Timberlake  spotify:track:1AWQoqb9bSvzTjaLralEkT   
    0     4                 Shaggy  spotify:track:1lzr43nnXAijIGYnCT8M8H   
    0     5                  Usher  spotify:track:0XUfyU2QviPAs6bxSpXYG4   
    0     6                  Usher  spotify:track:68vgtRHr7iZHpzGpon6Jlo   
    0     7     The Pussycat Dolls  spotify:track:3BxWKCI06eQ5Od8TY2JBeA   
    0     8        Destiny's Child  spotify:track:7H6ev70Weq6DdpZyyTmUXk   
    0     9                OutKast  spotify:track:2PpruBYCo4H7WOBJ7Q2EwM   
    0    10          Nelly Furtado  spotify:track:2gam98EZKrF9XuOkU13ApN   
    0    11        Jesse McCartney  spotify:track:4Y45aqo9QMa57rDsAJv40A   
    0    12        Jesse McCartney  spotify:track:1HwpWwa6bnqqRhK8agG4RS   
    0    13        Jesse McCartney  spotify:track:20ORwCJusz4KS2PbTPVNKo   
    0    14                 Cassie  spotify:track:7k6IzwMGpxnRghE7YosnXT   
    0    15                Omarion  spotify:track:1Bv0Yl01xBDZD4OQP93fyl   
    0    16          Avril Lavigne  spotify:track:4omisSlTk6Dsq2iQD7MA07   
    0    17            Chris Brown  spotify:track:7xYnUQigPoIDAMPVK79NEq   
    0    18                Beyoncé  spotify:track:6d8A5sAx9TfdeseDvfWNHd   
    0    19        Destiny's Child  spotify:track:4pmc2AxSEq6g7hPVlJCPyP   
    0    20            Sheryl Crow  spotify:track:215JYyyUnrJ98NK3KEwu6d   
    0    21    The Black Eyed Peas  spotify:track:0uqPG793dkDDN7sCUJJIVC   
    0    22       Bowling For Soup  spotify:track:19Js5ypV6JKn4DMExHQbGc   
    0    23         The Click Five  spotify:track:1JURww012QnWAw0zZXi6Aa   
    0    24            Chris Brown  spotify:track:7DFnq8FYhHMCylykf6ZCxA   
    0    25         Jonas Brothers  spotify:track:1TfAhjzRBWzYZ8IdUV3igl   
    0    26               Lil Mama  spotify:track:1Y4ZdPOOgCUhBcKZOrUFiS   
    0    27                Cascada  spotify:track:6MjljecHzHelUDismyKkba   
    0    28           Jason Derulo  spotify:track:67T6l4q3zVjC5nZZPXByU8   
    0    29                  Ne-Yo  spotify:track:34ceTg8ChN5HjrqiIYCn9Q   
...      ..                    ...                                   ...   
1   998  15               Bon Jovi  spotify:track:0J6mQxEZnlRt9ymzFntA6z   
    998  16                  Train  spotify:track:6mghCOaaSvrke0z1EUVUIf   
    998  17                  Train  spotify:track:4ocbH5ZJqajZIpUrqRUoue   
    998  18      Five For Fighting  spotify:track:5gBQ3x6u7DCvMW1QoP3CpG   
    998  19               The Fray  spotify:track:5fVZC9GiM4e8vu99W0Xf6J   
    998  20               The Fray  spotify:track:4IoYz8XqqdowINzfRrFnhi   
    998  21      Simon & Garfunkel  spotify:track:0iOZM63lendWRTTeKhZBSC   
    998  22                Journey  spotify:track:4bHsxqR3GMrXTxEPLuK5ue   
    998  23                Journey  spotify:track:71SvEDmsOwIWw1IozsZoMA   
    998  24              Tom Petty  spotify:track:5tVA6TkbaAH9QMITTQRrNv   
    998  25  Red Hot Chili Peppers  spotify:track:3d9DChrdc6BOeFsbrZ3Is0   
    998  26  Red Hot Chili Peppers  spotify:track:10Nmj3JCNoMeBQ87uw5j8k   
    998  27           Lukas Graham  spotify:track:7675gjlUZzneYiMrQ9Inx8   
    998  28        Louis Armstrong  spotify:track:29U7stRjqHU6rMiS8BfaI9   
    999   0              blink-182  spotify:track:2wMqxbWlfIIaVzpYtFG4HY   
    999   1              blink-182  spotify:track:1e1krgHCTUGDhhIgsRVHJ0   
    999   2      Breaking Benjamin  spotify:track:4BJyt25nburVwbnESDeIc7   
    999   3              Crossfade  spotify:track:6ZxrUCaynLgKSUudACOTwj   
    999   4                   Enya  spotify:track:6FLwmdmW77N1Pxb1aWsZmO   
    999   5                 Filter  spotify:track:2c5wml

# Make unique playlist ID's and split into train and test

In [258]:
unique_playlist_ids = np.unique(complete_playlists.index.values)
np.random.shuffle(unique_playlist_ids)

# 0.8 Test and 0.2 Train Split
train_idx_cutoff = int(0.8 * unique_playlist_ids.shape[0])
train_playlist_ids = unique_playlist_ids[0:train_idx_cutoff]
test_playlist_ids = unique_playlist_ids[train_idx_cutoff:unique_playlist_ids.shape[0]]

In [262]:
# Create 
def createSparseMatrix(train_playlist_ids, filter_method = "track_name", all_candidates):    
    rows_list = []
    for ids in train_playlist_ids:
        playlist_filter = complete_playlists.loc[ids][filter_method]
        ones_idx = np.where(np.isin(unique_filter, playlist_filter))[0]
        row = np.zeros(unique_filter.shape)
        row[ones_idx] = 1 / playlist_filter.shape[0]
        rows_list.append(row)
    train_sparse = np.vstack(rows_list)
    return train_sparse

SyntaxError: non-default argument follows default argument (<ipython-input-262-ced4a3014af6>, line 2)

In [261]:
train_sparse = createSparseMatrix(train_playlist_ids, "track_name", all_unique_songs)

In [265]:
compressed_sparse = sp.sparse.csr_matrix(train_sparse)
model_knn_compressed = NearestNeighbors(metric='cosine', 
                                        algorithm='brute', 
                                        n_neighbors=20, n_jobs=-1).fit(compressed_sparse)

In [ ]:
rows_list = []
val_list = []
# For each playlist in test playlists
for ids in test_playlist_ids:
    test_playlist = complete_playlists.loc[ids]
    
    # Split into test songs and validation songs
    # Could be better
    #np.random.shuffle(test_playlists)
    idx = np.random.choice(np.arange(len(test_playlist)), int(0.8 * len(test_playlist)), replace=False)
    test_idx = np.where(np.in1d(np.arange(len(test_playlist)), idx, invert=True))[0]
    songs_to_use = test_playlist["track_name"].values[idx]
    songs_to_evaluate = test_playlist.iloc[test_idx]
    val_list.append(songs_to_evaluate)
    
    # Make the matrix for test playlist
    ones_idx = np.where(np.in1d(all_unique_songs, songs_to_use))[0]
    row = np.zeros(all_unique_songs.shape)
    row[ones_idx] = 1 # / songs_to_use.shape[0]
    rows_list.append(row)
    
test_sparse = np.vstack(rows_list)
compressed_test_sparse = sp.sparse.csr_matrix(np.vstack(rows_list))
val_df = pd.concat(val_list)
test_indices = test_playlist_ids

In [270]:
def getSongPredictions(X, model, indices, candidate_playlists, top_n, col_names):
    distances, neighbors = model.kneighbors(X)
    predictions_list = []
    for i in range(X.shape[0]):
        best_candidates = neighbors[i][distances[i] != 1]
        best_distances = distances[i][distances[i] != 1]
        if best_candidates.size != 0:
            # get the place in candidate_playlists with the lowest distance
            candidate_idx = indices[best_candidates]
        else:
            # TODO: Change
            predictions_list.append(pd.DataFrame(columns = candidate_playlists.columns))
            continue
            
        candidates_df = candidate_playlists.loc[list(map(tuple, candidate_idx))]
        
        # transform the best_distances into a form that can be concatenated
        lengths = candidates_df.groupby(level=[0,1])["pos"].count().values
        

        candidates_df["distance"] = np.repeat(best_distances, lengths)
        
        # want only the songs that are not in the playlist
        not_in_train_df = candidates_df.iloc[np.where(np.in1d(candidates_df["track_name"].values, 
                                                      col_names[X[i].toarray()[0] != 0], invert=True))[0]]
        
        # drop duplicates
        no_duplicates_df = not_in_train_df.drop_duplicates("track_name")
        if no_duplicates_df.shape[0] < top_n:
            predictions_list.append(no_duplicates_df)
        else:
            predictions_list.append(no_duplicates_df.iloc[0:top_n])
    return predictions_list

In [271]:
predictions_list = getSongPredictions(compressed_test_sparse, model_knn_compressed, 
                                      train_playlist_ids, complete_playlists, 500, all_unique_songs)

# Evaluate

In [272]:
clicks = []
for i in range(compressed_test_sparse.shape[0]):
    predictions = predictions_list[i]
    idx = test_indices[i]
    val_songs = val_df.loc[tuple(idx)]
    positions_matching = np.where(np.in1d(predictions["track_name"].values, val_songs['track_name']))[0]
    # CLICKS: min pos / 10
    if positions_matching.size == 0:
        clicks.append(50)
    else:
        clicks.append(math.ceil(positions_matching[0] / 10))

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  """


In [274]:
clicks

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 19,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 6,
 50,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 17,
 0,
 0,
 0,
 50,
 0,
 0,
 0,
 0,
 50,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 37,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 50,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 2,
 2,
 0,
 0,
 24,
 0,
 0,
 0,
 0,
 0,
 0,
 50,
 7,
 0,
 1,
 0,
 0,
 50,
 0,
 0,
 0,
 0,
 0,
 21,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 50,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 14,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 41,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 50,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,


In [273]:
np.mean(clicks)

3.38

In [275]:
complete_playlists.loc[test_playlist_ids[0]]

pos               artist_name                             track_uri  \
fid pid                                                                       
1   210   0          Barenaked Ladies  spotify:track:2RQ3fWFZJJe4G5KkHEACww   
    210   1            Counting Crows  spotify:track:1tjKIXd7DYNklhVSgBxREi   
    210   2    Motion City Soundtrack  spotify:track:6DTLxkODqriIxX6Q4d4bOC   
    210   3              Atlas Genius  spotify:track:0wDEiGfq4pw7FAkYKpkwMo   
    210   4                       M83  spotify:track:1eyzqe2QqGZUmfcPZtrIyt   
    210   5                  Gorillaz  spotify:track:0d28khcov6AiegSCpG5TuT   
    210   6                Washed Out  spotify:track:4i13Vno2UPtn6kp2Ge9N2J   
    210   7              Matt and Kim  spotify:track:57i3M29DWoo7RDk0Tf0LZG   
    210   8                      Muse  spotify:track:3skn2lauGk7Dx6bVIt5DVj   
    210   9    Motion City Soundtrack  spotify:track:7N2DblLu8DuZ0seGYSg73v   
    210  10           Young the Giant  spotify:track:1UqhkbzB1kuFwt2iy4h29Q   
    210  11     Green River Ordinance  spotify:track:6uhkZFj3vzLY0FA9ONCM41   
    210  12               The Antlers  spotify:track:371oFaGHOqCmqRwx0XxEpo   
    210  13            Love and Theft  spotify:track:63w0QA1wiV7QhF9jeiHETF   
    210  14                     Gotye  spotify:track:4wCmqSrbyCgxEXROQE6vtV   
    210  15                Ben Howard  spotify:track:0Qw7qGuXlVBZLcfoQ17yZn   
    210  16                     B.o.B  spotify:track:28GUjBGqZVcAV4PHSYzkj2   
    210  17                     Gomez  spotify:track:6LGtr8FSW9stUCqTqzPHSu   
    210  18               Chiddy Bang  spotify:track:0O986IRZ2rL3qGH5QiD4ZO   
    210  19              The Outfield  spotify:track:4oDZ5L8izBals6jKBJDBcX   
    210  20                 Graffiti6  spotify:track:0vXdd32H6j8ggGapOHfan1   
    210  21  Stars of Track and Field  spotify:track:2727cNzjNBfVtjkDHLpuzr   
    210  22    The Chain Gang Of 1974  spotify:track:2LnrajrTTxWdUgLBdaGTpL   
    210  23                 Slim Thug  spotify:track:4hFV8xf1MD6Tmg2g0XltTY   
    210  24            Electric Guest  spotify:track:1tzPSule9WZ9B8SujHv8fp   
    210  25                AWOLNATION  spotify:track:7hsLKGEnoiNShdIGL6ws1f   
    210  26          Josh Abbott Band  spotify:track:35DciiuaU8Lut9EkrlP4wR   
    210  27                  Maroon 5  spotify:track:1XGmzt0PVuFgQYYnV2It7A   
    210  28                Kanye West  spotify:track:4qikXelSRKvoCqFcHLB2H2   
    210  29                     Usher  spotify:track:2U5NrHirVZusBLQlohTbnZ   
...      ..                       ...                                   ...   
    210  40              Delta Spirit  spotify:track:4Y3gLYlV5rD3fhirhWKtei   
    210  41             WALK THE MOON  spotify:track:3e0yTP5trHBBVvV32jwXqF   
    210  42                   J. Dash  spotify:track:5tqyZmF6uEoTkD6ja7KZjv   
    210  43               Danny Brown  spotify:track:1jqB1YzZaDrxYKaFqTDLUe   
    210  44             The Oh Hellos  spotify:track:2cMqgf2xZ6cMc0IyBHCl8x   
    210  45                 Wolf Gang  spotify:track:3Ufbnur2EwjNQoVKyoX1ue   
    210  46                 Wolf Gang  spotify:track:6QU4xwSi2X8np8qo7JJb34   
    210  47       Bombay Bicycle Club  spotify:track:4yYvWFUWMaKMXtoJKJnmZk   
    210  48                    Kimbra  spotify:track:15zUlY0ELQ9fTbmg0W01Qs   
    210  49               Mat Kearney  spotify:track:7otG2nsXUjnQq0EgxuBpD3   
    210  50           Ball Park Music  spotify:track:5UUEgckqHvJ7OB9Du04pxE   
    210  51                   Pusha T  spotify:track:34Fulx6Umr9LoA4UKdcjVP   
    210  52         Silversun Pickups  spotify:track:1aDLUzCyYpRXgrjwUWzV2X   
    210  53          Miracle Fortress  spotify:track:5qKCMOrqRhEKG9Zg8J3fFX   
    210  54                     JJAMZ  spotify:track:43SGRLrSS4m4b8YfSrwvNZ   
    210  55               Mat Kearney  spotify:track:0RUXHlYhA057fCOe2vqIqu   
    210  56                  Carousel  spotify:track:1P0JP5gsaDDMvJOB02dl7B   
    210  57               Nappy Roots  spotify:track:58NcF

In [277]:
val_df

pos               artist_name                             track_uri  \
fid pid                                                                        
1   511    5                    Khalid  spotify:track:25YvGDl2zSE0pH8jrMZ6aY   
    511    6                  The 1975  spotify:track:316r1KLN0bcmpr7TZcMCXT   
    511    7                     Drake  spotify:track:7hDc8b7IXETo14hHIHdnhd   
    511   17                    Khalid  spotify:track:3rtwdAFtZQiFu1WT0dUVjG   
    511   24             Martin Garrix  spotify:track:6jA8HL9i4QGzsj6fjoxp8Y   
    511   26                Wild Front  spotify:track:47Q3RLDX27XVeuQYicgLRj   
    511   33                      Russ  spotify:track:6qcuRcbz8DCLG6nG4or8Uo   
0   114    5             Willie Nelson  spotify:track:4vcDnnuHUmombz20IRkzRO   
    114    9             Willie Nelson  spotify:track:0VLKfcHGOn0x0fXZqJrX9o   
    114   10             Willie Nelson  spotify:track:5TtCm24ufNn1ME9dcM5Iwc   
    114   11             Willie Nelson  spotify:track:0ZXmbQXKbfXO8Cv9qxPdZu   
    114   22             Willie Nelson  spotify:track:4XqhtbkrbF9qX1Qa7iQTFH   
1   102    2             Damian Marley  spotify:track:7e6rBpGavwHi1QnoEP0JvF   
    102   12              Ziggi Recado  spotify:track:0wqjdq7nZUVdNOEdyu8xb7   
    102   14                   Red Rat  spotify:track:1uNIWPKOnJw92GPDcvaJY9   
    102   18                 Sean Paul  spotify:track:0k2GOhqsrxDTAbFFSdNJjT   
    102   20                 Sean Paul  spotify:track:4kOTDj4SiZPkW1p96WMDNv   
    102   25                   Lumidee  spotify:track:4c9VuqkKaey9u0IihbUKTt   
    102   29  Bob Marley & The Wailers  spotify:track:2mKsPUojh602HvSeNt04CB   
    102   30                Beenie Man  spotify:track:3j8EEMm9kbPLqByuaHmUmQ   
    102   35                    T.O.K.  spotify:track:05jHFnv6I9aOij4uipICJe   
    102   36     Tanto Metro & Devonte  spotify:track:00FU5uypELwgQuM9IbFC6G   
    102   39                    Serani  spotify:track:44gtrHC8pnRzjnQqdkRZPY   
    102   55                    T.O.K.  spotify:track:3dMJkWhv6WXKQVHhFw8kxP   
0   477    5           Various Artists  spotify:track:4H8rHV9KRBI79yubbgfyW8   
    477    8             Zion & Lennox  spotify:track:5nhIC014t6LRlY8qO5tWXx   
    477   11           New Year's Hits  spotify:track:6faCSbuEu5X2vmpxlx2JhM   
    178   14                 Fetty Wap  spotify:track:2d8JP84HNLKhmd6IYOoupQ   
    178   22                     Drake  spotify:track:11KJSRSgaDxqydKYiD2Jew   
    178   26              Lil Uzi Vert  spotify:track:2ANLarE8yHVsLWW21nj79M   
...      ...                       ...                                   ...   
    854  120            Montell Jordan  spotify:track:6b0mf7tkMei83pEeITIqIf   
    854  123                    Jaheim  spotify:track:38084tN66v4KHqiLpl9cN8   
    854  138               Keith Sweat  spotify:track:0KgbwF3wEBvPWPwwTLo9iR   
    854  139               Keith Sweat  spotify:track:24gxdUxufJ5eSamdYcPAKH   
    854  143               Jagged Edge  spotify:track:5nS5mDWAyce9gjD8crHcZX   
    854  144                    H-Town  spotify:track:6x53SJOV1PAZ8ZUwH4NFXQ   
    854  145                 Ruff Endz  spotify:track:1TYEfvGTYpdimygplYu0Hk   
    854  147               Boyz II Men  spotify:track:3l3xTXsUXeWlkPqzMs7mPD   
    854  149               Boyz II Men  spotify:track:5on5ilENckx5vV7d50yvIt   
    854  151       Ready For The World  spotify:track:1OlYvKsCBfbK3sD5fgalF2   
    854  156              Pretty Ricky  spotify:track:5y4TDQdYzlT4eoQIPOgNDz   
    854  162                J. Holiday  spotify:track:3nJjIYchBNkYm7B8XfJf5N   
    854  165                     Mario  spotify:track:5xV0Czdqefft6sPDqjmFBu   
    854  171                  DJ Snake  spotify:track:4OZEu0Vwq0RB2LAq14v99n   
    854  172               Skizzy Mars  spotify:track:6OUJdL8jcNzL3zj8WQ6UDw   
    854  177                  Ace Hood  spotify:track:0Quyy3FEdh7Il3d1x5qGXW   
    854  178                   Jeremih  spotify:track:2KqyIbbCPKhNfqqzUVq9H9   
    854  

In [207]:
complete_playlists[complete_playlists["track_uri"] == "spotify:track:11pfbduq7fbQxmXH03OKo7"]

pos artist_name                             track_uri  \
fid pid                                                          
0   362   19   Sam Smith  spotify:track:11pfbduq7fbQxmXH03OKo7   
    432   43   Sam Smith  spotify:track:11pfbduq7fbQxmXH03OKo7   
    433    8   Sam Smith  spotify:track:11pfbduq7fbQxmXH03OKo7   
    648   30   Sam Smith  spotify:track:11pfbduq7fbQxmXH03OKo7   
    919   42   Sam Smith  spotify:track:11pfbduq7fbQxmXH03OKo7   

                                    artist_uri       track_name  \
fid pid                                                           
0   362  spotify:artist:2wY79sveU1sp5g7SokKOiI  Not In That Way   
    432  spotify:artist:2wY79sveU1sp5g7SokKOiI  Not In That Way   
    433  spotify:artist:2wY79sveU1sp5g7SokKOiI  Not In That Way   
    648  spotify:artist:2wY79sveU1sp5g7SokKOiI  Not In That Way   
    919  spotify:artist:2wY79sveU1sp5g7SokKOiI  Not In That Way   

                                    album_uri  duration_ms          album_name  
fid pid                                                                         
0   362  spotify:album:2Jg7JZ0ZXOGje1bkq7CVgK       172452  In The Lonely Hour  
    432  spotify:album:2Jg7JZ0ZXOGje1bkq7CVgK       172452  In The Lonely Hour  
    433  spotify:album:2Jg7JZ0ZXOGje1bkq7CVgK       172452  In The Lonely Hour  
    648  spotify:album:2Jg7JZ0ZXOGje1bkq7CVgK       172452  In The Lonely Hour  
    919  spotify:album:2Jg7JZ0ZXOGje1bkq7CVgK       172452  In The Lonely Hour

In [177]:
predictions_list[4]

pos             artist_name                             track_uri  \
fid pid                                                                      
0   607    0      Corinne Bailey Rae  spotify:track:2l1BYvzTOKFKDLAXeQc3SE   
    607    1              Chet Faker  spotify:track:2xD7RvKFJl52xyds3wQFXQ   
    607    2          The Civil Wars  spotify:track:4zzi2eD2cEPpQ3a307mPPj   
    607    3                  Jungle  spotify:track:3g86N0SqyfxEVNMHlmW3RE   
    607    4      Family of the Year  spotify:track:3MkcVmqLSxHTkTFgczctbQ   
    607    5               Ben Folds  spotify:track:4hVDEHZgzxSdYkQl732TvW   
    607    6          Gary Clark Jr.  spotify:track:6dT6nC1IxBPBF71t4IFm5G   
    607    7           Chris Cornell  spotify:track:4iWm8M2rSRAkmZYFGs12Fx   
    607    8                Coldplay  spotify:track:7D0RhFcb3CrfPuTJ0obrod   
    607    9            When In Rome  spotify:track:48p5E25cFPanxuwCTmTpuL   
    607   10              Us The Duo  spotify:track:6KzL8s8GdRbd5Mal8yLz6R   
    607   11         Yeah Yeah Yeahs  spotify:track:2hxV5yGCr1nIp5bQBxXL13   
    607   12      Corinne Bailey Rae  spotify:track:7aFQa52bLiHwb3BXPbmKdZ   
    607   13        Sleeping At Last  spotify:track:0dHF8t7OIhgPele5WRUfWG   
    607   14                  Hozier  spotify:track:2tKOIBLJ2j384tnunmmKi7   
    607   15         Freedy Johnston  spotify:track:5aNAOwr2QoRbvyRG7ZVrhf   
    607   16         Yeah Yeah Yeahs  spotify:track:3zvHOVC4jikl6jORSr6OLE   
    607   17                    Beck  spotify:track:3arVrdpOPMgLZOztBr2jM6   
    607   18          Regina Spektor  spotify:track:3FvxvqQmMCZLsIzNvFQqhz   
    607   19            Matt and Kim  spotify:track:57i3M29DWoo7RDk0Tf0LZG   
    607   20        Butthole Surfers  spotify:track:1ng36571Iyov4HBxUClySn   
    607   21                   Adele  spotify:track:7rPLZ8Krm6CZIbraFUlnWZ   
    607   22               Cat Power  spotify:track:6Pi8GUiL7VqbPbU8IhHxyP   
    607   23                  Jungle  spotify:track:6Pu1G8OV5m38PkLQHyZfPp   
    607   24     Trampled By Turtles  spotify:track:06gmYLiwfegMk3yHx26vjB   
    607   25           Black Sabbath  spotify:track:3b7wNR330iKhql9dEdbNci   
    607   26     Death Cab for Cutie  spotify:track:4mCF3EBgGPSqmEm205KBAV   
    607   27              Aloe Blacc  spotify:track:1FsH6OZCKLWbBgqj8JcCvO   
    607   28  James Vincent McMorrow  spotify:track:2hSdHOOBLXNkh7zmmgoAn3   
    607   29            Jack Johnson  spotify:track:12jjuxN1gxlm29cqL5M6MW   
...      ...                     ...                                   ...   
    13     2              Rob Thomas  spotify:track:1Zx5Y46u3v32jRRIWtzQKN   
    13     3           Jeffrey James  spotify:track:3dFJn0eFwXGZipJMpDgFsW   
    13     5        Jasmine Thompson  spotify:track:2PeQP3wjJ0ogwbUr1X6tU1   
    13     6         Gabrielle Aplin  spotify:track:7pq5vONxrAy8T773nhkiW3   
    13     7         Christina Perri  spotify:track:6lanRgr6wXibZr8KgzXxBl   
    13     8      Molly Kate Kestner  spotify:track:66ZEsTGwfUCcslLyIDCxzR   
    13     9         Katy McAllister  spotify:track:0RJmnZ23Usaa07LPUe4mGb   
    13    10          Joseph Vincent  spotify:track:07ceQy7UhpNDA2WhEOJgPp   
    13    11            James Arthur  spotify:track:3ZdCBj1cbossYy1O40ZmLD   
    13    12                 Ruth B.  spotify:track:5hwh37sTi84MVhCBMWzhGE   
    272    1              Bruno Mars  spotify:track:6SKwQghsR8AISlxhcwyA9R   
    272    2               Sam Smith  spotify:track:0fioLzGM8ngbD1w6fMmm45   
    272    3              Bruno Mars  spotify:track:7BqBn9nzAq8spo5e7cZ0dJ   
    272    4                   B.o.B  spotify:track:59dLtGBS26x7kc0rHbaPrq   
    272    5              Katy Perry  spotify:track:0ktV2JoOsoTGURzKaZnjJL   
    272    6                Maroon 5  spotify:track:494OU6M7NOf4ICYb4zWCf5   
    272    7              John Mayer  spotify:track:0QTCTu0CXv4X1JEE4gNpGv   
    272    8           X Ambassadors  spotify:track:46GGxd8TVRt6FjUBfCavVT   
    272    9                  M

# MULTIPROCESSING

In [74]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logging.getLogger('requests').setLevel(logging.CRITICAL)
logger = logging.getLogger(__name__)


def main():
    ts = time()
    with Pool(4) as p:
        p.map(createSparseMatrix, playlist_list)
    logging.info('Took %s seconds', time() - ts)


if __name__ == '__main__':
    main()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  
2019-12-09 15:46:04,484 - root - INFO - Took 1.4989278316497803 seconds
